In [ ]:
!pip install IMDbPY

In [16]:
from imdb import IMDb
import networkx as nx

def initialize_graph():
    return nx.Graph(), IMDb()

def find_actors(ia, actor_name_1, actor_name_2):
    actor_1_search = ia.search_person(actor_name_1)
    actor_2_search = ia.search_person(actor_name_2)

    if not actor_1_search or not actor_2_search:
        print("Could not find one of the actors.")
        return None, None

    actor_1 = actor_1_search[0]
    actor_2 = actor_2_search[0]
    print(f"Found actors: {actor_1['name']} and {actor_2['name']}.")
    return actor_1, actor_2

def add_actor_connections(graph, ia, actor):
    # Ensure actor node is added with name
    graph.add_node(actor.personID, type='actor', name=actor['name'])

    ia.update(actor, 'filmography')
    movies = actor.get('filmography', {}).get('actor', []) + actor.get('filmography', {}).get('actress', [])
    for movie in movies:
        movie_id = "m" + movie.movieID  # Prefix with 'm' to differentiate from actor IDs
        graph.add_node(movie_id, type='movie', title=movie['title'])
        graph.add_edge(actor.personID, movie_id)

def build_connections(graph, ia, actors):
    for actor in actors:
        add_actor_connections(graph, ia, actor)

def find_path_and_print(graph, actor_1, actor_2):
    try:
        path = nx.shortest_path(graph, source=actor_1.personID, target=actor_2.personID)
        print_path_info(graph, path)
    except nx.NetworkXNoPath:
        print(f"No connection found between {actor_1['name']} and {actor_2['name']}.")

def print_path_info(graph, path):
    print("Connection path:")
    for i in range(len(path) - 1):
        node_a, node_b = path[i], path[i+1]
        if graph.nodes[node_a].get('type') == 'actor':
            actor_name = graph.nodes[node_a].get('name')
            movie_title = graph.nodes[node_b].get('title')
            print(f"{actor_name} appeared in '{movie_title}'")
        else:
            actor_name = graph.nodes[node_b].get('name')
            movie_title = graph.nodes[node_a].get('title')
            print(f"{actor_name} in '{movie_title}'")

if __name__ == "__main__":
    graph, ia = initialize_graph()
    actor_name_1 = input("Enter the name of the first actor: ")
    actor_name_2 = input("Enter the name of the second actor: ")

    actor_1, actor_2 = find_actors(ia, actor_name_1, actor_name_2)
    if actor_1 and actor_2:
        build_connections(graph, ia, [actor_1, actor_2])
        find_path_and_print(graph, actor_1, actor_2)


Enter the name of the first actor: Keith David
Enter the name of the second actor: Kevin Spacey
Found actors: Keith David and Kevin Spacey.
Connection path:
Keith David appeared in 'Tribeca'
Kevin Spacey in 'Tribeca'
